In [ ]:
note="k"

In [ ]:
# This one uses processed Ws inputs, so one does not have to input the array sizes manually
import qpms
from qpms.symmetries import  gen_hexlattice_Kpoint_svwf_rep, gen_hexlattice_Kpoint_svwf_rep_projectors
import numpy as np
from matplotlib import pyplot as plt
import scipy
from scipy import interpolate
from scipy.constants import e as eV, hbar, hbar as ℏ, c
ň = np.newaxis
import cmath

In [ ]:
evfactor =  (eV/hbar) /( 3e8 / 1e-6 )
evfactor * 1.38, evfactor * 1.5

In [ ]:
Wfile = '/m/phys/project/qd/Marek/superdeep2_cs/superdeep2broadlMax3_cs'
#Wfile = '/m/phys/project/qd/Marek/gammapoint_cs/gammapoint_lMax3_rough'
#force_lMax = 2
#nelem = force_lMax * (force_lMax+2)

In [ ]:
#wdata = wdata_long
wdata = qpms.loadWfile_processed(Wfile, lMax=3, midk_halfwidth=1)
lMax = wdata['lMax']
nelem = wdata['nelem']

In [ ]:
#wdata.keys(), wdata['nk'], wdata['ks'], wdata['Ws'].shape

In [ ]:
#### T-matrices from file
# Original Ag:
TMatrix_file = '/u/46/necadam1/unix/tmatrix-experiments/Cylinder/hexlasingPaper/cylinder_50nm_lMax3_long.TMatrix'
# Zero loss "Ag":
#TMatrix_file = '/u/46/necadam1/unix/tmatrix-experiments/Cylinder/hexlasingPaper/zeroloss/cylinder_50nm_lMax2_long.TMatrix'

TMatrices_orig, freqs_orig, freqs_weirdunits_orig, lMaxTM = qpms.loadScuffTMatrices(TMatrix_file, normalisation = 2, version = 'new')
tmtype="Scuff"

In [ ]:
# TODO synchronize the TM/W lMaxes better
TMatrices_orig = TMatrices_orig[...,:,0:nelem,:,0:nelem]
lMaxTM = lMax

In [ ]:
freqs_orig[0] / (eV/hbar), freqs_orig[-1] / (eV/hbar)

In [ ]:
#### Alternatively, Mie T-matrix
#lMaxTM = force_lMax
lMax = lMaxTM

nelem = lMax * (lMax + 2)
startω =1*eV/ℏ
stopω = 2*eV/ℏ
ωlist = np.linspace(startω, stopω, 200)


ε_b = 2.13
R = 40e-9

if False: # drude model
    ω_p = 9*eV/ℏ #9*eV/ℏ
    ε_inf = 4.6
    γ_p = 0#0.07*eV/ℏ
    
    ε_mlist = qpms.ε_drude(ε_inf, ω_p, γ_p, ωlist)
    tmtype="MieDrude"
else: # interpolated data
    permdata = np.loadtxt("/u/46/necadam1/unix/tmatrix-experiments/materialy/Au-Johnson-simple.dat")
    permfreqs = permdata[:,0]
    permvals = permdata[:,1] + permdata[:,2]*1j
    permdata_interp = interpolate.interp1d(permfreqs, permvals, axis = 0, kind='linear', fill_value='extrapolate')
    ε_mlist = permdata_interp(ωlist)
    tmtype="MieInterp"
    

RVlist = np.empty((lMax+1,)+ωlist.shape,dtype=np.complex_)
RHlist = np.empty((lMax+1,)+ωlist.shape,dtype=np.complex_)

for i in range(len(ωlist)):
    ω=ωlist[i]
    k_i = cmath.sqrt(ε_mlist[i])*ω/c
    k_e = cmath.sqrt(ε_b)*ω/c
    RH, RV, TH, TV = qpms.mie_coefficients(a=R, nmax=lMax, k_i=k_i, k_e=k_e, J_ext=1, J_scat=3)
    RVlist[:,i] = RV
    RHlist[:,i] = RH

y = np.arange(nelem)
my, ny = qpms.get_mn_y(lMax)

TMatrices_orig = np.zeros((RVlist.shape[1], 2, nelem, 2, nelem), dtype=complex)

for l in range(1,lMax+1):
    beg = l**2-1
    end = l*(l+2)
    TMatrices_orig[:,0,beg:end,0,beg:end] = -RVlist[l,:,ň,ň] * np.eye(2*l+1) # FIXME opposite sign?
    TMatrices_orig[:,1,beg:end,1,beg:end] = -RHlist[l,:,ň,ň] * np.eye(2*l+1)

freqs_orig = ωlist

In [ ]:
lMax = lMaxTM  # redundant
nelem = qpms.get_nelem(lMax) #redundant
nomega = wdata['nfreqs'] # 286
nk = wdata['nk'] #100
npart = wdata['npart']
wdata['freqs'][0]/(eV/hbar), wdata['freqs'][-1] / (eV/hbar)

In [ ]:
#symmetrize TMatrices
yfl = qpms.yflip_tyty(lMax)
xfl = qpms.xflip_tyty(lMax)
zfl = qpms.zflip_tyty(lMax)
TMatrix=TMatrices_orig
TMatrix_sym = (TMatrix + qpms.apply_ndmatrix_left(yfl,qpms.apply_ndmatrix_left(yfl, TMatrix, (-4,-3)),(-2,-1)))/2
TMatrix_sym = (TMatrix_sym + qpms.apply_ndmatrix_left(xfl,qpms.apply_ndmatrix_left(xfl, TMatrix_sym, (-4,-3)),(-2,-1)))/2
TMatrix_sym = (TMatrix_sym + qpms.apply_ndmatrix_left(zfl,qpms.apply_ndmatrix_left(zfl, TMatrix_sym, (-4,-3)),(-2,-1)))/2
rotN = 600

TMatrix_contribs = np.empty((rotN,)+TMatrix.shape, dtype=np.complex_)
for i in range(rotN):
    rotangle = 2*np.pi*i / rotN
    rot =  qpms.WignerD_yy_fromvector(lMax,np.array([0,0,rotangle]))
    rotinv = qpms.WignerD_yy_fromvector(lMax,np.array([0,0,-rotangle]))
    TMatrix_contribs[i] = qpms.apply_matrix_left(rot,qpms.apply_matrix_left(rotinv, TMatrix_sym, -3),-1)
TMatrix_sym = np.sum(TMatrix_contribs, axis=0) / rotN
TMatrix_sym[np.abs(TMatrix_sym) <= 1e-16] = 0

In [ ]:
# TODO process (interpolate and symmetrize) T-matrix here
TMatrices_interp = interpolate.interp1d(freqs_orig, TMatrix_sym, axis = 0, kind='linear', fill_value='extrapolate')


In [ ]:
Wmatrices = wdata['Ws'].reshape((nomega,nk,npart,2,nelem,npart,2,nelem))
#Wmatrices = (wdata_long['Ws']+0*wdata_short['Ws']).reshape((nomega,nk,npart,2,nelem,npart,2,nelem))
#TMatrices = TMatrix_sym.reshape((nomega,1,2,nelem,2,nelem))
ks = np.broadcast_to(wdata['ks'][ň,:,:], (nomega, nk, 2))
freqs = np.broadcast_to(wdata['freqs'][:,ň], (nomega,nk))

Wmatrices.shape, ks.shape, freqs.shape


In [ ]:
TMatrices_orig.shape, ks.shape, freqs.shape

# Full problem
TE_or_TM = None # just that the scripts below know...
allU = np.full((nomega,nk,npart*2*nelem,npart*2*nelem), np.nan, dtype=complex)
allV = np.full((nomega,nk,npart*2*nelem,npart*2*nelem), np.nan, dtype=complex)
allS = np.full((nomega,nk,npart*2*nelem), np.nan)
for omi in range(nomega):
    for ki in range(nk):
        W = Wmatrices[omi,ki]
        T = TMatrices_interp(freqs[omi,ki]) * 1
        TW = np.sum(
            T[...,ň,:,:,:,:,ň,ň,ň] * W[...,:,ň,ň,:,:,:,:,:]
        , axis = (-5,-4))
        iden = np.eye(npart*2*nelem)
        iden.shape = (npart,2,nelem,npart,2,nelem)
        finalMatrix = iden-TW
        try:
            allU[omi,ki], allS[omi,ki], allV[omi,ki] = scipy.linalg.svd(finalMatrix.reshape((npart*2*nelem,npart*2*nelem)), 
                                                                        lapack_driver= 'gesvd')
        except ValueError:
            pass
restype="Full"                

In [ ]:
# Z-symmetric problem
TE_or_TM = 0 # 0 is TE

TE_yz = np.arange(nelem)
TM_yz = TE_yz
my, ny = qpms.get_mn_y(lMax)
TE_NMz = (my + ny) % 2
TM_NMz = (1 - TE_NMz)
T_NMz = TE_NMz if TE_or_TM == 0 else TM_NMz
T_yz = TE_yz if TE_or_TM == 0 else TM_yz
y = np.arange(nelem)
yi = y[ň,ň,ň,:]
yj = y[ň,:,ň,ň]
ipart = np.arange(npart)
ti = np.arange(2)

allU = np.full((nomega,nk,npart*nelem,npart*nelem), np.nan, dtype=complex)
allV = np.full((nomega,nk,npart*nelem,npart*nelem), np.nan, dtype=complex)
allS = np.full((nomega,nk,npart*nelem), np.nan)
for omi in range(nomega):
    for ki in range(nk):
        W = Wmatrices[omi,ki][..., ipart[:,ň,ň,ň], T_NMz[ň,:,ň,ň],T_yz[ň,:,ň,ň], ipart[ň,ň,:,ň], T_NMz[ň,ň,ň,:],T_yz[ň,ň,ň,:]] # [Npart,2,nelem,Npart,2,nelem] -> [Npart,nelem,Npart,nelem]
        W[np.isclose(W,0, atol = 1e-10)] = 0
        T = TMatrices_interp(freqs[omi,ki])[...,T_NMz[:,ň],T_yz[:,ň],T_NMz[ň,:],T_yz[ň,:]]  #[2,nelem,2,nelem] -> [nelem,nelem]
        TW = np.sum(
            T[...,ň,:,:,ň,ň] * W[...,:,ň,:,:,:]
        , axis = (-3))
        iden = np.eye(npart*nelem)
        iden.shape = (npart,nelem,npart,nelem)
        finalMatrix = iden-TW
        try:
            allU[omi,ki], allS[omi,ki], allV[omi,ki] = scipy.linalg.svd(finalMatrix.reshape((npart*nelem,npart*nelem)), 
                                                                       # lapack_driver= 'gesvd'
                                                                       )
        except ValueError:
            pass

#theV = allV[:,:,sorder,:].reshape(nomega,nk,2,2,nelem)
allV_expanded = np.zeros((nomega,nk,npart*nelem,2,2,nelem),dtype=complex)
for t in (0,1):
    allV_expanded[:,:,:,:,t,T_NMz == t]  = allV.reshape((nomega,nk,npart*nelem,npart,nelem))[:,:,:,:,T_NMz == t]

if TE_or_TM == 0:
    restype="E-inplane"
elif TE_or_TM == 1:
    restype="M-inplane"
else:
    restype="WTF"

In [ ]:
allS.shape

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,10))
plt.pcolormesh(ks[:,:,1], freqs[:,:]*hbar/eV, allS[:,:,-1])
plt.colorbar()
plt.savefig("figs/lMax%d%s%s%s_disp.pdf"%(lMax,tmtype,restype,note))

In [ ]:
### Zde se vybírá ""číslo"" modu
psi = -1
reps =  gen_hexlattice_Kpoint_svwf_rep(lMax, psi)
projmats = gen_hexlattice_Kpoint_svwf_rep_projectors(lMax,psi)
for sorder in (-1,):#,-2,-3,-4,-5):
    # Change this for full / zsym solutions
    #theV = allV[:,:,sorder,:].reshape(nomega,nk,2,2,nelem)
    theV = allV_expanded[:,:,sorder,:,:,:]

    theVprojections_v1 = dict() # non-normalised projector (probably this one is correct)
    theVprojections_v2 = dict() # normalised projector
    theVprojections_v3 = dict() # normalised projector squared

    for repkey, rep in reps.items():
        genvec = np.ones((2,2,nelem))
        symproj = np.tensordot(rep, genvec, axes = ([-3,-2,-1],[0,1,2])) #[d,d,p,t,y]; old method
        projmat = projmats[repkey]
        #FIXME tenhle přísstup je možná taky blbě, já vlastně chci spíš možná sčítat čtverce z theV...
        theVprojections_pre = np.abs(np.sum(theV[:,:,ň,ň,:,:,:] * symproj, axis=(-3,-2,-1))) #[nomega,nk,d,d] # raději kvadrát???
        theVprojections_v1[repkey] = np.sum(theVprojections_pre, axis = (-2,-1))
        theVprojections_v2[repkey] = np.sum(theVprojections_pre / np.sum(symproj*symproj.conj(),axis=(-3,-2,-1)).real,(-2,-1))
        theV_halfproj = np.tensordot(theV.conjugate(),projmat, axes=([-3,-2,-1],[0,1,2])) 
        theVprojections_v3[repkey] = np.sum( theV_halfproj[:,:,:,:,:] * theV[:,:,:], axis = (-3,-2,-1)  ).real
        
        
                                                  

    if TE_or_TM is None:
        repkeylist = ['A1p', 'A1pp', 'A2p', 'A2pp', 'E1', 'E2']# list(reps.keys())
    elif TE_or_TM ==0:
        repkeylist = ['A1p', 'A2p', 'E1',]# list(reps.keys())
    elif TE_or_TM ==1:
        repkeylist = ['A1pp', 'A2pp', 'E2']# list(reps.keys())
    else:
        raise ValueError('TE_or_TM has wrong value (or has not been set?)')


    theVprojections_v1_arr = np.empty((nomega,nk,len(repkeylist)), dtype=float)
    theVprojections_v2_arr = np.empty((nomega,nk,len(repkeylist)), dtype=float)
    theVprojections_v3_arr = np.empty((nomega,nk,len(repkeylist)), dtype=float)
    for i in range(len(repkeylist)):
        theVprojections_v1_arr[:,:,i] = theVprojections_v1[repkeylist[i]]
        theVprojections_v2_arr[:,:,i] = theVprojections_v2[repkeylist[i]]
        theVprojections_v3_arr[:,:,i] = theVprojections_v3[repkeylist[i]]

    for key in repkeylist:
        plt.figure(figsize=(10,10))
        plt.pcolormesh(ks[:,:,1], freqs[:,:]*hbar/eV, theVprojections_v3[key], )
        plt.title(key)
        plt.colorbar()
        #plt.imshow(theVprojections_v3[key], origin = 'lower')

In [ ]:
theVprojections_v1_order = np.argsort(theVprojections_v1_arr, axis = -1)
theVprojections_v2_order = np.argsort(theVprojections_v2_arr, axis = -1)
theVprojections_v3_order = np.argsort(theVprojections_v3_arr, axis = -1)

In [ ]:
cmap = plt.get_cmap('jet', len(repkeylist))

plt.figure(figsize=(15,15))
plt.pcolormesh(ks[:,:,1], freqs[:,:]*hbar/eV, theVprojections_v1_order[:,:,0], cmap=cmap, vmin = -0.5, vmax = len(repkeylist)-0.5)
plt.colorbar()

#plt.imshow(theVprojections_v3_order[:,:,0], cmap=cmap, vmin = -0.5, vmax = len(repkeylist)-0.5, origin='lower')

In [ ]:
cmap = plt.get_cmap('jet', len(repkeylist))

plt.figure(figsize=(15,15))
plt.pcolormesh(ks[:,:,1], freqs[:,:]*hbar/eV, theVprojections_v3_order[:,:,1], cmap=cmap, vmin = -0.5, vmax = len(repkeylist)-0.5)
plt.colorbar()

#plt.imshow(theVprojections_v3_order[:,:,1], cmap=cmap, vmin = -0.5, vmax = len(repkeylist)-0.5, origin='lower')

In [ ]:
cmap = plt.get_cmap('jet', len(repkeylist))

plt.figure(figsize=(15,15))
plt.pcolormesh(ks[:,:,1], freqs[:,:]*hbar/eV, theVprojections_v1_order[:,:,-1], cmap=cmap, vmin = -0.5, vmax = len(repkeylist)-0.5)
plt.colorbar()

#plt.imshow(theVprojections_v3_order[:,:,-1], cmap=cmap, vmin = -0.5, vmax = len(repkeylist)-0.5, origin='lower')

In [ ]:
print(nk, nk//2)
sh = 0
plt.figure(figsize=(15,15))
xen = freqs[:,0]*hbar/eV
plt.plot(xen,theVprojections_v3['E1'][:,nk//2-sh],label = 'E1')
plt.plot(xen,theVprojections_v3['A2p'][:,nk//2-sh],label = 'A2p')
plt.plot(xen,theVprojections_v3['A1p'][:,nk//2-sh], label= 'A1p')
plt.plot(xen,allS[:,nk//2-sh,-1])
plt.plot(xen,allS[:,nk//2-sh,-2])
plt.legend()
plt.savefig("figs/lMax%d%s%spsi%d%s_Elines.pdf"%(lMax,tmtype,restype,psi,note))

In [ ]:

plt.figure(figsize=(15,15))
plt.plot(xen,theVprojections_v3['E2'][:,nk//2], label='E2')
plt.plot(xen,theVprojections_v3['A1pp'][:,nk//2], label='A1pp')
plt.plot(xen,theVprojections_v3['A2pp'][:,nk//2], label='A2pp')
plt.plot(xen,allS[:,nk//2,-1], label='Singular value')
plt.legend()
plt.savefig("figs/lMax%d%s%spsi%d%s_Mlines.pdf"%(lMax,tmtype,restype,psi,note))

In [ ]:
i=145
wdata['freqs_weirdunits'][i], wdata['freqs'][i]/eV*hbar